###This notebook contains some usual analysis for LPA
It only works for output format in pdb.

In [ ]:
import numpy as np
import pylab as plt
from lpa_pdb_diagnostics import *
from scipy.constants import e, c, m_e, epsilon_0
import os
import matplotlib
%matplotlib inline

Setting the directory path where the data are located. dir_path has to be correctly attributed before analysis.

In [ ]:
#If reading several files
num_res = 2
dir_path = np.empty(num_res,dtype="S100")
nxplambda = np.array([2,4])

for index in xrange(num_res):
    dir_path[index] = "/Volumes/Orsay/gilles_Nitrogen/gilles_Nitrogen0.01_%d/" %nxplambda[index]   +"data/"   

Setting the path for the results:

In [ ]:
res_path = config.result_path

#Laser - Plasma Parameters
Here we define some of the laser plasma parameters.

In [ ]:
# Some definitions of the laser plasma parameters
lambda0 = 0.8e-6
w0 = 2*np.pi*c/lambda0
laser_waist = 17e-6
plasma_dens = 4.307e24
plasma_length = 4.5e-3
wp = np.sqrt(plasma_dens * e**2/ (epsilon_0 * m_e))
lambda_plasma = 2*np.pi*c/wp
circ_m = 1

#Numerical Parameters
Here we define the necessary numerical parameters for the analysis

In [ ]:
#longitudinal direction 
zmax = 2*lambda0
zmin = zmax - 4*lambda_plasma
w_size = zmax - zmin
nzplambda = np.ones(num_res)*20
Nz = np.array(w_size*nzplambda/lambda0).astype(int)
dz = w_size/Nz

#transverse direction
xmax = 100e-6
xmin = 0 # for cylindrical coordinates
Nx = np.array((xmax - xmin)*nxplambda/lambda0).astype(int)
dx = (xmax - xmin)/Nx

#time (valid only for circ simulation)
circ_coeffs = [ 0.2105, 1.0, 3.5234, 8.5104, 15.5059, 24.5037 ]

if circ_m < len(circ_coeffs):
    circ_alpha = circ_coeffs[circ_m]
else:
    circ_alpha = circ_m**2 - 0.4

cdt = 1./np.sqrt((1 + circ_alpha)/dx**2 + 1./dz**2)

#Generate values for file reading

In [ ]:
inf = 0
sup = 0
period_int = 1000
period_ext = 1000
val = []
longitudinal_position = []

for i, x in enumerate(cdt):
    val.append(values( inf, sup, period_int, period_ext, plasma_length/x ))
    longitudinal_position.append(np.array(val[i])*cdt[i])

We create an array of file names that we analyze.

In [ ]:
field = [[] for i in xrange(num_res)]
N5 = [[] for i in xrange(num_res)]
N6 = [[] for i in xrange(num_res)]
N7 = [[] for i in xrange(num_res)]
H = [[] for i in xrange(num_res)]

# Initialize file names
for k in xrange(num_res):
    for i, v in enumerate(val[k]):
        field[k].append(dir_path[k] + "fields%06d.pdb" %v)
        N5[k].append(dir_path[k] + "N5%06d.pdb" %v)
        N6[k].append(dir_path[k] + "N6%06d.pdb" %v)
        N7[k].append(dir_path[k] + "N7%06d.pdb" %v)
        H[k].append(dir_path[k] + "H%06d.pdb" %v)

#Reading files

##Multiple files reading

####Laser $a_0$, Beam spectrum, Big Picture

In [ ]:
l_particle = False

#preparing some empty arrays
a0 = [[] for i in xrange(num_res)]
z_a0 = [[] for i in xrange(num_res)]
emitx = [[] for i in xrange(num_res)]
emity = [[] for i in xrange(num_res)]
deltaE = [[] for i in xrange(num_res)]
deltaEE = [[] for i in xrange(num_res)]
divx = [[] for i in xrange(num_res)]
divy = [[] for i in xrange(num_res)]
charge = [[] for i in xrange(num_res)]
ROI_peak = [[] for i in xrange(num_res)]
ROI_peak_left = [[] for i in xrange(num_res)]
ROI_peak_right = [[] for i in xrange(num_res)]
long_pos = [[] for i in xrange(num_res)]
E_peak = [[] for i in xrange(num_res)]
z_wstd = [[] for i in xrange(num_res)]
x_wstd = [[] for i in xrange(num_res)]


for k in xrange(num_res):
    last = len(val[k])
    first  = 0 #int(1.2e-3/cdt[k]/period_int) #round off to the nearest 1000
    for index in range( first, last ):
        
        long_pos[k].append(val[k][index]*cdt[k])
        f = FieldInstant( field[k][index], np.pi/2, 
                     quantities= ["E", "zfield"] )
        a, z = f.laser_a0(w0)

        # Store values in arrays
        a0[k].append(a)
        z_a0[k].append(z)

        if l_particle:
            #p_H = ParticleInstant( H[index], quantities = [ "Weight", "Position", "Momentum"] )
            p_N6 = ParticleInstant( N6[k][index], quantities = [ "Weight", "Position", "Momentum"] )
            p_N7 = ParticleInstant( N7[k][index], quantities = [ "Weight", "Position", "Momentum"] )
            qdict = p_N6.get_qdict()

            #Select particles
            #cp_H = p_H.select( gamma_threshold = [50,800] )
            cp_N6 = p_N6.select( gamma_threshold = [50,800] )
            cp_N7 = p_N7.select( gamma_threshold = [50,800] )

            # Concatenate particles
            ck_all_particles = quant_concatenate([cp_N6,cp_N7], keep_object_name = True ) 
            #keeping the information on the species
            c_all_particles = quant_concatenate([cp_N6, cp_N7])

            # Analyzing beam spectrum
            energy, dQdE = beam_spectrum(val[k][index], 
                                 ck_all_particles[qdict["gamma"]], 
                                 ck_all_particles[qdict["w"]], lwrite=True,
                                 leg = ["N6","N7","Sum"])

            if energy is not None:
                t_energy = energy[-1]
                t_dQdE = dQdE[-1]

                Ipeak, Epeak, Cpeak, ROI_by_peak = beam_peak( t_energy, t_dQdE)

                if Ipeak is not None:
                    peak = (Ipeak[-1], Epeak[-1], Cpeak[-1])
                    t_deltaE , t_deltaEE = beam_energy_spread( t_energy, t_dQdE, peak = peak)

                else:
                    t_deltaE , t_deltaEE = beam_energy_spread( t_energy, t_dQdE )

                deltaE[k].append( t_deltaE )
                deltaEE[k].append( t_deltaEE )
                E_peak[k].append( Epeak )

            if energy is not None and ROI_by_peak:
                print "You have chosen particles situated between %g MeV and %g MeV. " \
                        %( ROI_by_peak[-1][0], ROI_by_peak[-1][1] )
                
                ROI_peak_left[k].append(ROI_by_peak[-1][0])
                ROI_peak_right[k].append(ROI_by_peak[-1][-1])
                # Conversion from MeV to gamma arb units
                ROI_gamma = [[ROI_by_peak[i][j]/0.511 for j in xrange(len(ROI_by_peak[0]))] \
                             for i in xrange(len(ROI_by_peak))]

                # Selection of particles
                cPN6 = p_N6.select( gamma_threshold = ROI_gamma[-1] )
                cPN7 = p_N7.select( gamma_threshold = ROI_gamma[-1] )

                ck_all_particles = quant_concatenate([cPN6,cPN7], keep_object_name= True) 
                #keeping the information on the species
                c_all_particles = quant_concatenate([cPN6,cPN7])

                ROI_peak[k].append( ROI_gamma[-1] )

            # Analysing big picture
            N_laser = f.normalizedField( w0, "laser")
            N_wake = f.normalizedField( wp, "wake")
            bigPicture( val[k][index], c_all_particles[qdict["z"]], c_all_particles[qdict["gamma"]],
               c_all_particles[qdict["w"]], f.zfield, N_wake, N_laser, lwrite= True )
            
            # Analyzing beam length
            t_z_wstd = wstd(c_all_particles[qdict["z"]], c_all_particles[qdict["w"]] ) 
            z_wstd[k].append(  t_z_wstd )
            t_x_wstd = wstd(c_all_particles[qdict["x"]], c_all_particles[qdict["w"]] ) 
            x_wstd[k].append( t_x_wstd )
        
            # Analyzing beam emittance
            t_emitx = beam_emittance( val[k][index],  c_all_particles, qdict, "x" ) 
            t_emity = beam_emittance( val[k][index],  c_all_particles, qdict, "y" )

            # Store values in arrays
            emitx[k].append( t_emitx )
            emity[k].append( t_emity )

            # Analyzing beam divergence
            t_divx  = beam_divergence (c_all_particles, qdict, "x")
            t_divy  = beam_divergence (c_all_particles, qdict, "y")

            # Store values in arrays
            divx[k].append( t_divx )
            divy[k].append( t_divy )

            # Analyze beam charge
            t_charge = beam_charge(c_all_particles[qdict["w"]])

            # Store values in array
            charge[k].append( t_charge )

##Saving laser $a_0$ values and beam properties in  in files.

In [ ]:
gname = ["nxplambda2", "nxplambda4"]
qname = ["z", "a0" ]
f = FileWriting(qname, "a0_hres", groups = gname)
stacked_data = np.stack(( z_a0, a0 ), axis=1)
f.write(stacked_data, np.shape(stacked_data),  attrs = ["m", "arb. unit" ])

In [ ]:
if l_particle:
    gname = ["nxplambda2", "nxplambda4"]
    qname = ["val", "deltaE", "deltaEE", "emitx", "emity", "divx", 
             "divy", "charge", "ROI_peak_left", "ROI_peak_right", "z_std", "x_std"]
    f = FileWriting(qname, "beam_properties_Nitrogen_hres", groups = gname)
    stacked_data = np.stack(( long_pos, deltaE, deltaEE, emitx, emity, 
                             divx, divy, charge, ROI_peak_left, ROI_peak_right, z_wstd, x_wstd), axis=1)
    f.write(stacked_data, np.shape(stacked_data), 
             attrs = [ "m", "MeV", "%", "m.rad", "m.rad", "rad", "rad", 
                      "C", "arb. units", "arb. units", "m", "m"])

##Plot $a_0$

In [ ]:
if 'inline' in matplotlib.get_backend():
    fig, ax = plt.subplots( dpi=150 )
else:
    fig, ax = plt.subplots( figsize=(10,8) )

fig.patch.set_facecolor('white')
ax.plot(z_a0, a0, linewidth = 2)

ax.set_xlabel(r"$\mathrm{z\,(m)}$")
ax.set_ylabel(r"$\mathrm{a_0}$")
ax.set_ylim(0.0, 1.1*max(a0))
ax.xaxis.set_tick_params(width=2, length = 8)
ax.yaxis.set_tick_params(width=2, length = 8)
font = {'family':'sans-serif'}
plt.rc('font', **font)

fig.savefig(res_path + "a0_hres.pdf")